In [17]:
import numpy as np
import pandas as pd
import tensorflow as tf

from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model

In [18]:
base_path = "//home/roger/code/benchmarking_deeplearning/"
data_folder = base_path + "data"
model_folder = base_path + "model"

In [19]:
cifar100 = tf.keras.datasets.cifar100

In [20]:
train, test = cifar100.load_data()

x_train, y_train = train
x_train = x_train.astype(np.float16)

x_test, y_test = test
x_test  = x_test.astype(np.float16)

In [21]:
train_ds = (tf.data.Dataset.from_tensor_slices((x_train, y_train))
    .shuffle(10000).batch(32))
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

In [22]:
class test_CNN(Model):
    def __init__(self):
        super(test_CNN, self).__init__()
        self.conv1 = Conv2D(32, 3, activation='relu')
        self.flatten = Flatten()
        self.d1 = Dense(128, activation='relu')
        self.d2 = Dense(100)

    def call(self, x):
        x = self.conv1(x)
        x = self.flatten(x)
        x = self.d1(x)
        return self.d2(x)

In [23]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.Adam()
model = test_CNN()

In [24]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [25]:
@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
        # training=True is only needed if there are layers with different
        # behavior during training versus inference (e.g. Dropout).
        predictions = model(images, training=True)
        loss = loss_object(labels, predictions)
        gradients = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))

        train_loss(loss)
        train_accuracy(labels, predictions)


In [26]:
@tf.function
def test_step(images, labels):
    # training=False is only needed if there are layers with different
    # behavior during training versus inference (e.g. Dropout).
    predictions = model(images, training=False)
    t_loss = loss_object(labels, predictions)

    test_loss(t_loss)
    test_accuracy(labels, predictions)


In [27]:
EPOCHS = 5

for epoch in range(EPOCHS):
    # Reset the metrics at the start of the next epoch
    train_loss.reset_states()
    train_accuracy.reset_states()
    test_loss.reset_states()
    test_accuracy.reset_states()

    for images, labels in train_ds:
        train_step(images, labels)

    for test_images, test_labels in test_ds:
        test_step(test_images, test_labels)

    print(
        f'Epoch {epoch + 1}, '
        f'Loss: {train_loss.result()}, '
        f'Accuracy: {train_accuracy.result() * 100}, '
        f'Test Loss: {test_loss.result()}, '
        f'Test Accuracy: {test_accuracy.result() * 100}'
    )


Epoch 1, Loss: 7.851195812225342, Accuracy: 0.8560000061988831, Test Loss: 4.605212688446045, Test Accuracy: 1.0
Epoch 2, Loss: 4.605669975280762, Accuracy: 0.8259999752044678, Test Loss: 4.605998992919922, Test Accuracy: 1.0099999904632568
Epoch 3, Loss: 4.605868339538574, Accuracy: 0.8679999709129333, Test Loss: 4.605704307556152, Test Accuracy: 1.0
Epoch 4, Loss: 4.60590934753418, Accuracy: 0.8700000047683716, Test Loss: 4.605215072631836, Test Accuracy: 1.0
Epoch 5, Loss: 4.605522632598877, Accuracy: 0.8720000386238098, Test Loss: 4.605223655700684, Test Accuracy: 1.0
